# All of Us lipids phenotypes

In this notebook we review and explore the available All of Us data for lipids phenotypes.

Note that the corresponding UKB phentypes are:
  * https://biobank.ndph.ox.ac.uk/showcase/field.cgi?id=30690
  * https://biobank.ndph.ox.ac.uk/showcase/field.cgi?id=30760
  * https://biobank.ndph.ox.ac.uk/showcase/field.cgi?id=30780
  * https://biobank.ndph.ox.ac.uk/showcase/field.cgi?id=30870

TODOs
* use lower and upper bound cutoffs appropriate for each measurement
* other issues to decide
  * timing of AoU measurements - which to use
    * right now its is using the most recent
    * another suggestion was to use the maximum value per participant
  * whether to incorporate the measurements with a missing unit concept id so that we can work with a larger portion of the cohort

# Setup 

<div class="alert alert-block alert-warning">
    <b>Cloud Environment</b>: This notebook was written for use on the All of Us Workbench. It runs fine on the default Cloud Environment. 
</div>

<div class="alert alert-block alert-success">
    <b>Most of the code in this notebook</b> came from the menu <kbd>Snippets -> All of Us R and SQL snippets</kbd>.
See also <a href="https://aousupporthelp.zendesk.com/hc/en-us/articles/360040043132-What-are-code-snippets-">What are code snippets?</a>
</div>


In [ ]:
lapply(c('viridis', 'ggthemes', 'skimr', 'fuzzyjoin'),
       function(pkg_name) { if(! pkg_name %in% installed.packages()) { install.packages(pkg_name)} } )
lapply(c('IRanges'),
       function(pkg_name) { if(! pkg_name %in% installed.packages()) { BiocManager::install(pkg_name)} } )

library(viridis)    # A nice color scheme for plots.
library(ggthemes)   # Common themes to change the look and feel of plots.
library(scales)     # Graphical scales map data to aesthetics in plots.
library(skimr)      # Better summaries of data.
library(lubridate)  # Date library from the tidyverse.
library(bigrquery)  # BigQuery R client.
library(tidyverse)  # Data wrangling packages.

In [ ]:
## BigQuery setup.
BILLING_PROJECT_ID <- Sys.getenv('GOOGLE_PROJECT')
# Get the BigQuery curated dataset for the current workspace context.
CDR <- Sys.getenv('WORKSPACE_CDR')

WORKSPACE_BUCKET <- Sys.getenv('WORKSPACE_BUCKET')

## Plot setup.
theme_set(theme_bw(base_size = 16)) # Default theme for plots.

#' Returns a data frame with a y position and a label, for use annotating ggplot boxplots.
#'
#' @param d A data frame.
#' @return A data frame with column y as max and column label as length.
get_boxplot_fun_data <- function(df) {
  return(data.frame(y = max(df), label = stringr::str_c('N = ', length(df))))
}

In [ ]:
## ---------------[ CHANGE THESE AS NEEDED] ---------------------------------------
COHORT_QUERY <- str_glue('SELECT person_id FROM `{CDR}.person`')  # Default to all participants.
MEASUREMENT_OF_INTEREST <- '(?i)cholesterol|hdl|ldl|triglyceride'

# Review measurements of interest 

In [ ]:
measurements_of_interest_summary_df <- bq_table_download(bq_project_query(
    BILLING_PROJECT_ID, page_size = 25000,
    query = str_glue('
-- Compute summary information for our measurements of interest for our cohort.
--
-- PARAMETERS:
--   MEASUREMENT_OF_INTEREST: a case-insensitive string, such as "hemoglobin", to be compared
--                            to all measurement concept names to identify those of interest

WITH
  --
  -- Use a case insensitive string to search the measurement concept names of those
  -- measurements we do have in the measurements table.
  --
  labs_of_interest AS (
  SELECT
    measurement_concept_id,
    measurement_concept.concept_name AS measurement_name,
    unit_concept_id,
    unit_concept.concept_name AS unit_name
  FROM
    `{CDR}.measurement`
  LEFT JOIN `{CDR}.concept` AS measurement_concept
  ON measurement_concept.concept_id = measurement_concept_id
  LEFT JOIN `{CDR}.concept` AS unit_concept
  ON unit_concept.concept_id = unit_concept_id
  WHERE
    REGEXP_CONTAINS(measurement_concept.concept_name, r"(?i){MEASUREMENT_OF_INTEREST}")
  GROUP BY
    measurement_concept_id,
    unit_concept_id,
    measurement_concept.concept_name,
    unit_concept.concept_name
)
  --
  -- Summarize the information about each measurement concept of interest that our
  -- prior query identified.
  --
SELECT
  measurement_name AS measurement,
  IFNULL(unit_name, "NA") AS unit,
  COUNT(1) AS N,
  COUNTIF(value_as_number IS NULL
    AND (value_as_concept_id IS NULL
      OR value_as_concept_id = 0)) AS missing,
  MIN(value_as_number) AS min,
  MAX(value_as_number) AS max,
  AVG(value_as_number) AS avg,
  STDDEV(value_as_number) AS stddev,
  APPROX_QUANTILES(value_as_number, 4) AS quantiles,
  COUNTIF(value_as_number IS NOT NULL) AS num_numeric_values,
  COUNTIF(value_as_concept_id IS NOT NULL
      AND value_as_concept_id != 0) AS num_concept_values,
  COUNTIF(operator_concept_id IS NOT NULL) AS num_operators,
  IF(src_id = "PPI/PM", "PPI", "EHR") AS measurement_source,
  measurement_concept_id,
  unit_concept_id
FROM
  `{CDR}.measurement`
INNER JOIN
 labs_of_interest USING(measurement_concept_id, unit_concept_id)
LEFT JOIN
  `{CDR}.measurement_ext` USING(measurement_id)
WHERE
  person_id IN ({COHORT_QUERY})
GROUP BY
  measurement_concept_id,
  measurement_name,
  measurement_source,
  unit_concept_id,
  unit_name
ORDER BY
  N DESC

')))

print(skim(measurements_of_interest_summary_df))

In [ ]:
head(measurements_of_interest_summary_df, n = 20)

# Retrieve and plot a most recent measurement of interest

This query only returns the most recent measurement per person.

In [ ]:
## ---------------[ CHANGE THIS AS NEEDED] ---------------------------------------
row_of_interest <- 1

(MEASUREMENT_NAME <- measurements_of_interest_summary_df$measurement[row_of_interest])
(MEASUREMENT_CONCEPT_ID <- measurements_of_interest_summary_df$measurement_concept_id[row_of_interest])
(UNIT_NAME <- measurements_of_interest_summary_df$unit[row_of_interest])
(UNIT_CONCEPT_ID <- measurements_of_interest_summary_df$unit_concept_id[row_of_interest])

In [ ]:
most_recent_measurement_of_interest_df <- bq_table_download(bq_project_query(
    BILLING_PROJECT_ID, page_size = 25000,
    query = str_glue('
-- Return row level data for a measurement, limited to only the most recent result per person in our cohort.
--
-- PARAMETERS:
--   MEASUREMENT_CONCEPT_ID: for example 3000963  # Hemoglobin
--   UNIT_CONCEPT_ID: for example 8636            # gram per liter

WITH
  --
  -- Retrieve participants birthdate and gender.
  --
persons AS (
  SELECT
    person_id,
    birth_datetime,
    concept_name AS gender
  FROM
    `{CDR}.person`
  LEFT JOIN `{CDR}.concept` ON concept_id = gender_concept_id),
  --
  -- Retrieve the row-level data for our measurement of interest. Also compute
  -- a new column for the recency rank of the measurement per person, a rank of
  -- of 1 being the most recent lab result for that person.
  --
measurements AS (
  SELECT
    person_id,
    measurement_id,
    measurement_concept_id,
    unit_concept_id,
    measurement_date,
    measurement_datetime,
    measurement_type_concept_id,
    operator_concept_id,
    value_as_number,
    value_as_concept_id,
    range_low,
    range_high,
    ROW_NUMBER() OVER (PARTITION BY person_id
                       ORDER BY measurement_date DESC,
                                measurement_datetime DESC,
                                measurement_id DESC) AS recency_rank

  FROM
    `{CDR}.measurement`
  WHERE
    measurement_concept_id = {MEASUREMENT_CONCEPT_ID}
    AND unit_concept_id = {UNIT_CONCEPT_ID}
    AND person_id IN ({COHORT_QUERY}))
  --
  -- Lastly, JOIN all this data together so that we have the birthdate, gender and site for each
  -- measurement, retaining only the most recent result per person.
  --
SELECT
  persons.*,
  src_id,
  measurements.* EXCEPT(person_id, measurement_id, recency_rank)
FROM
  measurements
LEFT JOIN
  persons USING (person_id)
LEFT JOIN
  `{CDR}.measurement_ext` USING (measurement_id)
WHERE
  recency_rank = 1
ORDER BY
  person_id,
  measurement_id

')))

print(skim(most_recent_measurement_of_interest_df))

In [ ]:
# Uncomment the line below to see row level data.
#head(most_recent_measurement_of_interest_df)

In [ ]:
# This plot assumes that most_recent_measurement_of_interest.sql has been run.
options(repr.plot.height = 12, repr.plot.width = 16)

most_recent_measurement_of_interest_df %>%
    # Get rid of nonsensical outliers.
    filter(value_as_number < 1000) %>%
    filter(value_as_number > 0) %>% # Get rid of nonsensical outliers.
    mutate(age_at_measurement = year(as.period(interval(start = birth_datetime, end = measurement_date)))) %>%
    # Exclude measurements taken during childhood.
    filter(age_at_measurement > 20) %>%
    ggplot(aes(x = cut_width(age_at_measurement, width = 10, boundary = 0), y = value_as_number)) +
    geom_boxplot() +
    stat_summary(fun.data = get_boxplot_fun_data, geom = 'text', size = 4,
                 position = position_dodge(width = 0.9), vjust = -0.8) +
#    scale_y_log10() +  # Uncomment if the data looks skewed.
    coord_flip() +
#    facet_wrap(~ gender, nrow = length(unique(most_recent_measurement_of_interest_df$gender))) +
    xlab('age') +
    ylab(str_glue('{UNIT_NAME}')) +
    labs(title = str_glue('Most recent {MEASUREMENT_NAME} measurement\nper person, by age'),
         caption = 'Source: All Of Us Data')

# Retrieve and plot several most recent measurements of interest

This query only returns the most recent measurement per person of each of several measurements.

In [ ]:
## ---------------[ CHANGE THIS AS NEEDED] ---------------------------------------
rows_of_interest <- seq(1, 4)

In [ ]:
CONCEPTS_OF_INTEREST <- measurements_of_interest_summary_df[rows_of_interest, ] %>%
    select(measurement_concept_id, unit_concept_id) %>%
    pmap(function (measurement_concept_id, unit_concept_id) {
        str_glue('(measurement_concept_id = {measurement_concept_id} AND unit_concept_id = {unit_concept_id})')
    }) %>%
    str_c(collapse = ' OR ')

CONCEPTS_OF_INTEREST

In [ ]:
multiple_most_recent_measurements_of_interest_df <- bq_table_download(bq_project_query(
    BILLING_PROJECT_ID, page_size = 25000,
    query = str_glue('
-- Return row level data for a measurement, limited to only the most recent result per person in our cohort.
--
-- PARAMETERS:
--   MEASUREMENT_CONCEPT_ID: for example 3000963  # Hemoglobin
--   UNIT_CONCEPT_ID: for example 8636            # gram per liter

WITH
  --
  -- Retrieve participants birthdate and gender.
  --
persons AS (
  SELECT
    person_id,
    birth_datetime,
    concept_name AS gender
  FROM
    `{CDR}.person`
  LEFT JOIN `{CDR}.concept` ON concept_id = gender_concept_id),
  --
  -- Retrieve the row-level data for our measurement of interest. Also compute
  -- a new column for the recency rank of the measurement per person, a rank of
  -- of 1 being the most recent lab result for that person.
  --
measurements AS (
  SELECT
    person_id,
    measurement_id,
    measurement_concept_id,
    unit_concept_id,
    measurement_date,
    measurement_datetime,
    measurement_type_concept_id,
    operator_concept_id,
    value_as_number,
    value_as_concept_id,
    range_low,
    range_high,
    ROW_NUMBER() OVER (PARTITION BY person_id, measurement_concept_id, unit_concept_id
                       ORDER BY measurement_date DESC,
                                measurement_datetime DESC,
                                measurement_id DESC) AS recency_rank

  FROM
    `{CDR}.measurement`
  WHERE
    ({CONCEPTS_OF_INTEREST})
    AND person_id IN ({COHORT_QUERY}))
  --
  -- Lastly, JOIN all this data together so that we have the birthdate, gender and site for each
  -- measurement, retaining only the most recent result per person.
  --
SELECT
  persons.*,
  CONCAT(measurement_concept.concept_name, " [", unit_concept.concept_name, "]") AS title,
  measurements.* EXCEPT(person_id, measurement_id, recency_rank)
FROM
  measurements
LEFT JOIN
  persons USING (person_id)
LEFT JOIN `{CDR}.concept` AS measurement_concept
  ON measurement_concept.concept_id = measurement_concept_id
LEFT JOIN `{CDR}.concept` AS unit_concept
  ON unit_concept.concept_id = unit_concept_id
WHERE
  recency_rank = 1
ORDER BY
  person_id,
  measurement_id

')))

dim(multiple_most_recent_measurements_of_interest_df)

## AoU Tabular summary 

In [ ]:
(aou_summary <- multiple_most_recent_measurements_of_interest_df %>%
    group_by(title) %>%
    summarize(
        count = n(),
        missing = sum(is.na(value_as_number)),
        median = median(value_as_number, na.rm = TRUE),
        mean = mean(value_as_number, na.rm = TRUE),
        stddev = sd(value_as_number, na.rm = TRUE)
    ))

In [ ]:
(ukb_summary <- read_csv(pipe('gsutil cat $WORKSPACE_BUCKET/data/ukb_lipids_summary.csv')) %>%
    rename(title='measurement'))

## Combined AoU + UKB tabular summary

In [ ]:
(combined_summary <- rbind(ukb_summary,
      aou_summary %>% mutate(data = 'AoU most recent measurement per person')
     ))

## Ploted by age group - full cohort

In [ ]:
options(repr.plot.height = 20, repr.plot.width = 16)

multiple_most_recent_measurements_of_interest_df %>%
    filter(value_as_number > 0) %>% # Get rid of nonsensical outliers.
    mutate(age_at_measurement = year(as.period(interval(start = birth_datetime, end = measurement_date)))) %>%
    # Exclude measurements taken during childhood.
    filter(age_at_measurement > 20) %>%
    ggplot(aes(x = cut_width(age_at_measurement, width = 10, boundary = 0), y = value_as_number)) +
    geom_boxplot() +
    stat_summary(fun.data = get_boxplot_fun_data, geom = 'text', size = 4,
                 position = position_dodge(width = 0.9), vjust = -0.8) +
    scale_y_continuous(breaks = scales::pretty_breaks(n = 10)) +
#    scale_y_log10(breaks = scales::pretty_breaks(n = 10)) +  # Uncomment if the data looks skewed.
    coord_flip() +
    facet_wrap(~ title, nrow = length(unique(multiple_most_recent_measurements_of_interest_df$title)), scales = 'free_x') +
    xlab('age') +
    labs(title = str_glue('Most recent measurement per person, by age'),
         caption = 'Source: All Of Us Data')

## Retrieve ids for participants in the alpha2 genomics release

In [ ]:
participants_with_genomic_data <- read_csv(
    pipe(str_glue('gsutil cat {WORKSPACE_BUCKET}/data/researchIDsAlpha2Release_04272021.txt')),
    col_names = c('person_id')
)

dim(participants_with_genomic_data)

## Ploted by age group - alpha2 genomics cohort

In [ ]:
options(repr.plot.height = 20, repr.plot.width = 16)

multiple_most_recent_measurements_of_interest_df %>%
    filter(value_as_number > 0) %>% # Get rid of nonsensical outliers.
    filter(person_id %in% participants_with_genomic_data$person_id) %>%    
    mutate(age_at_measurement = year(as.period(interval(start = birth_datetime, end = measurement_date)))) %>%
    # Exclude measurements taken during childhood.
    filter(age_at_measurement > 20) %>%
    ggplot(aes(x = cut_width(age_at_measurement, width = 10, boundary = 0), y = value_as_number)) +
    geom_boxplot() +
    stat_summary(fun.data = get_boxplot_fun_data, geom = 'text', size = 4,
                 position = position_dodge(width = 0.9), vjust = -0.8) +
    scale_y_continuous(breaks = scales::pretty_breaks(n = 10)) +
#    scale_y_log10(breaks = scales::pretty_breaks(n = 10)) +  # Uncomment if the data looks skewed.
    coord_flip() +
    facet_wrap(~ title, nrow = length(unique(multiple_most_recent_measurements_of_interest_df$title)), scales = 'free_x') +
    xlab('age') +
    labs(title = str_glue('Most recent measurement per person, by age'),
         caption = 'Source: All Of Us Data')

# Export to CSV

## Export available lipids measurements summary

In [ ]:
# This snippet assumes that you run setup first

# This code saves your dataframe into a csv file in a "data" folder in Google Bucket

# Replace df with THE NAME OF YOUR DATAFRAME
my_dataframe <- measurements_of_interest_summary_df %>%
    mutate(quantiles = str_c(quantiles))

# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename <- 'lipids_measurements_of_interest_summary.csv'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# store the dataframe in current workspace
write_excel_csv(my_dataframe, destination_filename)

# Get the bucket name
my_bucket <- Sys.getenv('WORKSPACE_BUCKET')

# Copy the file from current workspace to the bucket
system(paste0("gsutil cp ./", destination_filename, " ", my_bucket, "/data/"), intern=T)

# Check if file is in the bucket
system(paste0("gsutil ls ", my_bucket, "/data/*.csv"), intern=T)


## Export UKB + AoU comparison

In [ ]:
# This snippet assumes that you run setup first

# This code saves your dataframe into a csv file in a "data" folder in Google Bucket

# Replace df with THE NAME OF YOUR DATAFRAME
my_dataframe <- combined_summary

# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename <- 'combined_lipids_summary.csv'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# store the dataframe in current workspace
write_excel_csv(my_dataframe, destination_filename)

# Get the bucket name
my_bucket <- Sys.getenv('WORKSPACE_BUCKET')

# Copy the file from current workspace to the bucket
system(paste0("gsutil cp ./", destination_filename, " ", my_bucket, "/data/"), intern=T)

# Check if file is in the bucket
system(paste0("gsutil ls ", my_bucket, "/data/*.csv"), intern=T)


## Export row-level measurements data

In [ ]:
# This snippet assumes that you run setup first

# This code saves your dataframe into a csv file in a "data" folder in Google Bucket

# Replace df with THE NAME OF YOUR DATAFRAME
my_dataframe <- multiple_most_recent_measurements_of_interest_df

# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename <- 'most_recent_lipids_measurements.csv'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# store the dataframe in current workspace
write_excel_csv(my_dataframe, destination_filename)

# Get the bucket name
my_bucket <- Sys.getenv('WORKSPACE_BUCKET')

# Copy the file from current workspace to the bucket
system(paste0("gsutil cp ./", destination_filename, " ", my_bucket, "/data/"), intern=T)

# Check if file is in the bucket
system(paste0("gsutil ls ", my_bucket, "/data/*.csv"), intern=T)


# Provenance 

In [ ]:
devtools::session_info()